## Neuronales Netz Basisfunktionen in Numpy

In [ ]:
import numpy as np
import os
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
sns.set_style("whitegrid")
from sklearn.metrics import accuracy_score

In [ ]:
# Architektur des neuronalen Netzwerks
# wichtig ist, dass die aufeinander folgenden Layer die gleiche Größe haben
# KANN MAN VERÄNDERN
NN_ARCHITECTURE = [
    {"input_dim": 2, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 50, "activation": "relu"},
    {"input_dim": 50, "output_dim": 25, "activation": "relu"},
    {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"},
]

In [ ]:
# Das Netz muss am Anfang initialisiert werden 
# und das am besten nicht mit 0, weil dann kommt
# immer 0 raus
def init_layers(nn_architecture, seed = 99):
    # Wahl eines Random Seed, zur Reproduzierbarkeit 
    np.random.seed(seed)
    # Anzahl der Layer
    number_of_layers = len(nn_architecture)
    # Speicher der Gewichte
    params_values = {}
    
    # Schleife über alle Layer
    for idx, layer in enumerate(nn_architecture):
        # Wir starten bei 1 (ausnahmweise)
        layer_idx = idx + 1
        
        # Input und output Dimensionen
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        # Initialisieren der Gewichte W und der Vektoren b
        # für jeden Layer mit normalverteilen Werten
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
        
    return params_values

In [ ]:
# Aktivierungsfunktionen und deren Ableitung
def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

In [ ]:
# Vorwärtsdurchlauf eines einzelnen Layers
def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    # Berechnung des Inputs mit den Gewichten und dem Bias
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    # Auswahl der Aktivierungsfunktion
    if activation == "relu":
        activation_func = relu
    elif activation == "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    # Rückgabe des neuen Inputs für das nächste Layer
    return activation_func(Z_curr), Z_curr

In [ ]:
# Volle Berechnung des Vorwärtsdurchlauf durch das gesamte Netz
def full_forward_propagation(X, params_values, nn_architecture):
    # temporärer Speicher für den Rückwärtslauf
    memory = {}
    # Erster Input
    A_curr = X
    
    # Schleife über alle Layer
    for idx, layer in enumerate(nn_architecture):
        # wir starten wieder bei 1 
        layer_idx = idx + 1
        # Setze den Output der letzten Iteration auf den aktuellen Wert
        A_prev = A_curr
        
        # Auswahl der Aktivierungsfunktion
        activ_function_curr = layer["activation"]
        # Auswahl der entsprechenden Gewichte
        W_curr = params_values["W" + str(layer_idx)]
        # Auswahl der entsprechenden Biases
        b_curr = params_values["b" + str(layer_idx)]
        # Vorwärtsdurchlauf des aktuellen Layers
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        # Speichern alles wichtige für später
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    # Rückgabe des Outputs und der gespeicherten Werte für später
    return A_curr, memory

In [ ]:
# Berechung der Kostenfunktion, also wie gut ist denn unser Model 
def get_cost_value(Y_hat, Y):
    # Anzahl der Beispiele
    m = Y_hat.shape[1]
    # Berechnung der Kosten anhand der logistischen Regression
    cost = -1 / m * (np.dot(Y, np.log(Y_hat).T) + np.dot(1 - Y, np.log(1 - Y_hat).T))
    return np.squeeze(cost)

In [ ]:
# Hilfsfunktion um die vorhergesagten Wahrscheinlichkeiten in die Klassen umzuwandeln
def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_

In [ ]:
# Berechnung der Genauigkeit unseres Models
def get_accuracy_value(Y_hat, Y):
    Y_hat_ = convert_prob_into_class(Y_hat)
    return (Y_hat_ == Y).all(axis=0).mean()

In [ ]:
# Rückwärtsdurchlauf durch ein einzelnes Layer, um die Parameter zu verbessern
def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    # Anzahl der Beispiele
    m = A_prev.shape[1]
    
    # Auswahl der Aktivierungsfunktion
    if activation == "relu":
        backward_activation_func = relu_backward
    elif activation == "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    # Berechnung der Ableiung der Aktivierungsfuktion
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    
    # Ableitung der Gewichtsmatrix W
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    # Ableitung des Biasvektors b
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    # Ableitung der Inputmatrix A_prev
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

In [ ]:
# Rückwärtsdurchlauf durch das gesamte Netz
def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    
    # Anzahl der Beispiele
    m = Y.shape[1]
    # Sicherstellen, dass alles die gleiche Dimension hat
    Y = Y.reshape(Y_hat.shape)
    
    # Erster Schritt des Gradientenabstiegs
    dA_prev = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat));
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        # Wir starten immernoch bei 1
        layer_idx_curr = layer_idx_prev + 1
        # Auswahl der aktuellen Aktivierungsfunktion
        activ_function_curr = layer["activation"]

        # Initialisung der entsprechenden Werte
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]

        # Rückwärtsdurchlauf durch das aktuelle Layer
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        # Speichern der entsprchenden Gradienten 
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
    
    return grads_values

In [ ]:
# Funktion um die Gewichte der Layer zu verändern
def update(params_values, grads_values, nn_architecture, learning_rate):

    # Schleiche über alle Layer
    for layer_idx, layer in enumerate(nn_architecture, 1):
        # Update über die Gewichte
        params_values["W" + str(layer_idx)] -= learning_rate * grads_values["dW" + str(layer_idx)]        
        params_values["b" + str(layer_idx)] -= learning_rate * grads_values["db" + str(layer_idx)]

    return params_values

In [ ]:
# Funktion die alles was wir vorher gebaut haben zusammenführt
# SEED IN init_layers VERÄNDERN
def train(X, Y, X_test, Y_test,nn_architecture, epochs, learning_rate, verbose=False):
    # Initialisierung des Netzes
    params_values = init_layers(nn_architecture, 2)
    # Initialisierung, um das Training später anzuschauen 
    cost_history = []
    accuracy_history = []
    cost_test_history = []
    accuracy_test_history = [] 
    
    # Schleife über eine vorher festgelegte Anzahl an Durchlöufen (epochs)
    for i in range(epochs):
        # kompletter Vorwärtsschritt
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        
        # Berechnung von Kosten und Genauigkeit und speichern
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)

        Y_hat_test, _ = full_forward_propagation(X_test, params_values, nn_architecture)
        cost_test = get_cost_value(Y_hat_test, Y_test)
        cost_test_history.append(cost_test)
        accuracy_test = get_accuracy_value(Y_hat_test, Y_test)
        accuracy_test_history.append(accuracy_test)
        
        # kompletter Rückwärtsschritt um den Gradienten (Ableitung) zu berechnen
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        # Update der Parameter im Netz
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)

        # Ausdrucken der aktuellen Werte des Trainings
        if(i % 50 == 0):
            if(verbose):
                print("Iteration: {:05} - cost: {:.5f} - accuracy: {:.5f}".format(i, cost, accuracy))
            
    return params_values, [cost_history, accuracy_history, cost_test_history, accuracy_test_history]

## Visualisierung der Daten

In [ ]:
# Größe des Datensatzes
# N_SAMPLES VERÄNDERN
N_SAMPLES = 1000
# Größe des Testsets
# TEST_SIZE VERÄNDERN
TEST_SIZE = 0.1

In [ ]:
# Generierung des Datansatzes und split in test und train set
X, y = make_moons(n_samples = N_SAMPLES, noise=0.2, random_state=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [ ]:
# Hilfsfunktion um die Daten zu Visualisiern
def make_plot(X, y, plot_name, file_name=None, XX=None, YY=None, preds=None, dark=False):
    if (dark):
        plt.style.use('dark_background')
    else:
        sns.set_style("whitegrid")
    plt.figure(figsize=(16,12))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if(XX is not None and YY is not None and preds is not None):
        plt.contourf(XX, YY, preds.reshape(XX.shape), 25, alpha = 1, cmap=cm.Spectral)
        plt.contour(XX, YY, preds.reshape(XX.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    if(file_name):
        plt.savefig(file_name)
        plt.close()

In [ ]:
# Visualisiung des gesamten Datensatzes
make_plot(X, y, "Dataset")

## Training des Models und Visualisung der Ergebnisse

In [ ]:
# Training des Neuronalen Netzes
# 10000 Gradientenschritte
# Lernrate 0.01
epochs = 10000
learning_rate = 0.01
params_values, history = train(np.transpose(X_train), np.transpose(y_train.reshape((y_train.shape[0], 1))),np.transpose(X_test), np.transpose(y_test.reshape((y_test.shape[0], 1))), NN_ARCHITECTURE, epochs, learning_rate)

In [ ]:
# Vorwärtsdurchlauf auf den Testdaten mit den gelernten Gewichten
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), params_values, NN_ARCHITECTURE)

In [ ]:
# Check der Genauigkeit
acc_test = get_accuracy_value(Y_test_hat, np.transpose(y_test.reshape((y_test.shape[0], 1))))
print("Test set accuracy: {:.2f}".format(acc_test))

In [ ]:
def plot_learn_data(data1,data2,name="Cost"):
    plt.figure(figsize=(16,12))
    axes = plt.gca()
    axes.set(xlabel="$Epochen$", ylabel=name)
    plt.title(name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    plt.plot(data1)
    plt.plot(data2)
    axes.legend(['Train Set', 'Test Set'])

In [ ]:
# Plot der Kostenfunktion
plot_learn_data(history[0],history[2])
# Plot Genauigkeit
plot_learn_data(history[1],history[3],name="Accuracy")

In [ ]:
# Hilfsvariablen für die Visualisierung
GRID_X_START = -1.5
GRID_X_END = 2.5
GRID_Y_START = -1.0
GRID_Y_END = 2

# Erstellen eines Gitteres für das Bild
grid = np.mgrid[GRID_X_START:GRID_X_END:100j,GRID_X_START:GRID_Y_END:100j]
grid_2d = grid.reshape(2, -1).T
XX, YY = grid

In [ ]:
# Vorwärtsdurchlauf des Netzes mit dem gelernten Gewichten auf dem Gitter
prediction_probs_numpy, _ = full_forward_propagation(np.transpose(grid_2d), params_values, NN_ARCHITECTURE)
prediction_probs_numpy = prediction_probs_numpy.reshape(prediction_probs_numpy.shape[1], 1)
# Visualisierung der gelernten Gewichte und dem Test set
make_plot(X_test, y_test, "NumPy Model mit gelernten Gewichten", file_name=None, XX=XX, YY=YY, preds=prediction_probs_numpy)

## Visualisiung der initialen Gewichte

In [ ]:
init_values = init_layers(NN_ARCHITECTURE, 2)
prediction_probs_numpy, _ = full_forward_propagation(np.transpose(grid_2d), init_values, NN_ARCHITECTURE)
prediction_probs_numpy = prediction_probs_numpy.reshape(prediction_probs_numpy.shape[1], 1)
# Visualisierung der initialen Gewichte und dem Test set
make_plot(X_test, y_test, "NumPy Model mit initialen Gewichten", file_name=None, XX=XX, YY=YY, preds=prediction_probs_numpy)